In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)


In [ ]:
# df = pd.read_csv('raw_data/precision-experiment-2022-11-09T14_30_34.775Z (2).csv')
df = pd.read_csv("raw_data/precision-experiment-2023-01-24T17_15_59.068Z.csv")

df.head(5)


In [ ]:
df[~df['events'].isna()]

In [ ]:
def calculate_and_plot_errors(
    df, TRIAL_TAG, first_sample=0, max_plots=5, screen_res=(1920, 1080), verbose=True
):
    """_summary_

    Args:
        df (Pandas DataFrame): Data
        TRIAL_TAG (str): Type of trial (`validation-stimulus` or `fixation-stimulus`)
        first_sample (int, optional): First sample fo evaluate. Useful for filtering. Defaults to 0.
        max_plots (int, optional): Max number of plots. Defaults to 5.
        screen_res (tuple, optional): Screen Resolution of experiment. Defaults to (1920, 1080).

    Returns:
        pd.DataFrame: Analyzed data with columns:
            "trials": trials
            "presented_point": presented_points
            "time_between_samples_mean": time_between_samples_mean
            "time_between_samples_std": time_between_samples_std
            "sampling_rate_mean": sampling_rate_mean
            "sampling_rate_std": sampling_rate_std
            "last_time_sample": last_time_sample
            "horizontal_errors_pxs_mean": horizontal_errors_pxs_mean,
            "horizontal_errors_pxs_std": horizontal_errors_pxs_std,
            "vertical_errors_pxs_mean": vertical_errors_pxs_mean,
            "vertical_errors_pxs_std": vertical_errors_pxs_std,
            "total_errors_pxs_mean": total_errors_pxs_mean,
            "total_errors_pxs_std": total_errors_pxs_std
    """
    trials = []
    presented_points = []
    time_between_samples_mean = []
    time_between_samples_std = []
    sampling_rate_mean = []
    sampling_rate_std = []
    last_time_sample = []
    horizontal_errors_pxs_mean = []
    horizontal_errors_pxs_std = []
    vertical_errors_pxs_mean = []
    vertical_errors_pxs_std = []
    total_errors_pxs_mean = []
    total_errors_pxs_std = []

    center_x = df[df["trial-tag"] == TRIAL_TAG]["center_x"].iloc[0]
    center_y = df[df["trial-tag"] == TRIAL_TAG]["center_y"].iloc[0]

    nv = df[df["trial-tag"] == TRIAL_TAG]["validation-id"].values
    xv = df[df["trial-tag"] == TRIAL_TAG]["start-x"].values + center_x
    yv = df[df["trial-tag"] == TRIAL_TAG]["start-y"].values + center_y

    k = 0
    for d in df[df["trial-tag"] == TRIAL_TAG]["webgazer_data"].map(eval):

        xs = []
        ys = []
        ts = []
        for i in d:
            xs.append(i["x"])
            ys.append(i["y"])
            ts.append(i["t"])

        trials.append(nv[k])
        presented_points.append((xv[k], yv[k]))
        time_between_samples_mean.append(np.mean(np.diff(ts)))
        time_between_samples_std.append(np.std(np.diff(ts)))
        sampling_rate_mean.append(np.mean(1000 / np.diff(ts)))
        sampling_rate_std.append(np.std(1000 / np.diff(ts)))
        last_time_sample.append(max(ts))

        xs = np.array(xs)
        ys = np.array(ys)
        ts = np.array(ts)

        xs = xs[ts > first_sample]
        ys = ys[ts > first_sample] 
        ts = ts[ts > first_sample]

        ex = abs(xs - xv[k])
        ey = abs(ys - yv[k])
        ee = np.sqrt(ex**2 + ey**2)

        horizontal_errors_pxs_mean.append(np.mean(ex))
        horizontal_errors_pxs_std.append(np.std(ex))
        vertical_errors_pxs_mean.append(np.mean(ey))
        vertical_errors_pxs_std.append(np.std(ey))
        total_errors_pxs_mean.append(np.mean(ee))
        total_errors_pxs_std.append(np.std(ee))

        gs = gridspec.GridSpec(2, 4)
        gs.update(wspace=2)
        ax1 = plt.subplot(
            gs[0, :2],
        )
        ax1.set_title("Gaze estimation")
        ax1.scatter(xv[k], yv[k], c="c")
        ax1.vlines(screen_res[0] / 2, 0, screen_res[1], "k")
        ax1.hlines(screen_res[1] / 2, 0, screen_res[0], "k")
        colored_plot = ax1.scatter(xs, ys, c=ts)
        plt.colorbar(colored_plot, ax=ax1)
        ax1.scatter(xv[k], yv[k], c="b")
        ax1.set_xlim(0, screen_res[0])
        ax1.set_ylim(0, screen_res[1])

        ax2 = plt.subplot(gs[0, 2:])
        ax2.set_title("X coordinate over time")
        ax2.plot(ts, xs, "k.")
        ax2.set_ylim(0, center_x * 2)
        ax2.hlines(center_x, 0, max(ts), "k")

        ax3 = plt.subplot(gs[1, 1:3])
        ax3.set_title("y coordinate over time")
        ax3.plot(ts, ys, "k.")
        ax3.set_ylim(0, center_y * 2)
        ax3.hlines(center_y, 0, max(ts), "k")

        if verbose:
            plt.show()
            print("Horizontal error (pxs) = %.0f +- %.0f " % (np.mean(ex), np.std(ex)))
            print("Vertical error (pxs) = %.0f +- %.0f " % (np.mean(ey), np.std(ey)))
            print("Total error (pxs) = %.0f +- %.0f " % (np.mean(ee), np.std(ee)))
            print("Validation point (%d): %d, %d" % (nv[k], xv[k], yv[k]))
            print(
                "Time between samples (ms) = %.0f +- %.0f "
                % (np.mean(np.diff(ts)), np.std(np.diff(ts)))
            )
            print(
                "Sampling rate (Hz) = %.0f +- %.0f "
                % (np.mean(1000 / np.diff(ts)), np.std(1000 / np.diff(ts)))
            )
            print("Last time sample = %d" % max(ts))

        print("k:", k)
        k += 1
        if k == max_plots:
            break

    return pd.DataFrame(
        {
            "trials": trials,
            "presented_point": presented_points,
            "time_between_samples_mean": time_between_samples_mean,
            "time_between_samples_std": time_between_samples_std,
            "sampling_rate_mean": sampling_rate_mean,
            "sampling_rate_std": sampling_rate_std,
            "last_time_sample": last_time_sample,
            "horizontal_errors_pxs_mean": horizontal_errors_pxs_mean,
            "horizontal_errors_pxs_std": horizontal_errors_pxs_std,
            "vertical_errors_pxs_mean": vertical_errors_pxs_mean,
            "vertical_errors_pxs_std": vertical_errors_pxs_std,
            "total_errors_pxs_mean": total_errors_pxs_mean,
            "total_errors_pxs_std": total_errors_pxs_std,
        }
    )

In [ ]:
TRIAL_TAG = 'validation-stimulus'
# TRIAL_TAG = "fixation-stimulus"  # Es el momento donde aparece la cruz de fijacion

df_res = calculate_and_plot_errors(df, TRIAL_TAG,first_sample=300, max_plots=10, verbose=True)
print(f"Error: {df_res['total_errors_pxs_mean'].mean():.2f} +- {df_res['total_errors_pxs_mean'].std():.2f}")

In [ ]:
print(f"Error: {df_res['total_errors_pxs_mean'].mean():.2f} +- {df_res['total_errors_pxs_mean'].std():.2f}")

In [ ]:
print(f"Error: {df_res['total_errors_pxs_mean'].mean():.2f} +- {df_res['total_errors_pxs_mean'].std():.2f}")